<a href="https://colab.research.google.com/github/twyeh/highenergy/blob/main/5_Extending.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# flavio tutorial

## Part 5: Extending flavio

### Adding an observable: photon polarization in $B\to K\pi\pi\gamma$

$$\lambda_\gamma = \frac{|G_L|^2 - |G_R|^2}{|G_L|^2 + |G_R|^2}$$

$$G_L = C_7^\text{eff} + \ldots, \qquad G_L = C_7' + \ldots $$

$\ldots$ refer to non-factorizable hadronic contributions - let's ignore them for simplicity

Writing a function that takes a `WilsonCoefficients` instance and a dictionary of parameter values as input

In [ ]:
import flavio

def ll_lgamma(wc_obj, par_dict):
    scale = flavio.config['renormalization scale']['bvgamma']
    wc_dict = flavio.physics.bdecays.wilsoncoefficients.wctot_dict(wc_obj, 'bsee', scale, par_dict)
    delta_C7 = flavio.physics.bdecays.matrixelements.delta_C7(
        par=par_dict, wc=wc_dict, q2=0, scale=scale, qiqj='bs')
    a = {}
    GL = abs(wc_dict['C7eff_bs'] + delta_C7)**2
    GR = abs(wc_dict['C7effp_bs'])**2
    return (GL**2 - GR**2) / (GL**2 + GR**2)

Defining the `Observable` and `Prediction` instances

In [ ]:
obs = 'lambda_gamma'
flavio.classes.Observable(obs)
flavio.classes.Prediction(obs, ll_lgamma);

In [ ]:
flavio.sm_prediction('lambda_gamma')

In [ ]:
wc = flavio.WilsonCoefficients()
wc.set_initial({'C7p_bs': 0.25}, 4.8)
flavio.np_prediction('lambda_gamma', wc)

## Adding a new parameter

In [ ]:
flavio.classes.Parameter('my_fudge_factor')
flavio.default_parameters.set_constraint('my_fudge_factor', '0 +- 0.2')

In [ ]:
flavio.default_parameters.get_central('my_fudge_factor')

In [ ]:
[flavio.default_parameters.get_random_all()['my_fudge_factor'] for i in range(5)]

## Adding observables that depend on new operators

In principle, any observable where NP enters via local operators can be added to flavio:

- $D$ mixing & decays
- Non-leptonic $B$ decays
- Charged lepton flavour violation
- $(g-2)_\ell$
- Electric dipole moments
- Electroweak precision tests (via SMEFT)

### Extending the operator basis

To extend the operator basis, the additional operators have to be define in the WCxf basis; See [wcxf.github.io](https://wcxf.github.io).

One of the next release will also allow to define *observables* themselves in terms of other WCxf EFTs or bases (e.g. SMEFT).